In [1]:
import kagglehub
import os
from pathlib import Path

os.environ["KAGGLEHUB_CACHE"] = str(Path.cwd() / "data" / "kagglehub")

# Download latest version
path = kagglehub.dataset_download("marcocavaco/scraped-job-descriptions")

print("Path to dataset files:", path)

Path to dataset files: /Users/socialaistudio/Code/hiring_agent/data/kagglehub/datasets/marcocavaco/scraped-job-descriptions/versions/1


In [3]:
import pandas as pd
df = pd.read_csv(path + "/JD_data.csv", index_col=0)
df.head()

,ISCO,major_job,job,position,location,description
0,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist id54315,"Villigen PSI, Aargau",[' You have an academic degree in physics or e...
1,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,Geneva,[' Be in charge of the development of applicat...
2,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist (BE-ABP-LNO-2021-122-LD)...,Geneva,[' Contribute to the maintenance and developme...
3,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Medical Devices Physicist,"Newton, Cambridgeshire",[' Agency: Newton Colmore Consulting Reference...
4,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Fluidics Physicist,Cambridge,[' Agency: Newton Colmore Consulting Reference...


In [4]:
print(df['major_job'].unique(), len(df['major_job'].unique()))
print(df.ISCO.unique(), len(df.ISCO.unique()))
print(df['job'].value_counts())

['SCIENCE AND ENGINEERING PROFESSIONALS'
 'BUSINESS AND ADMINISTRATION ASSOCIATE PROFESSIONALS'
 'TEACHING PROFESSIONALS' 'HEALTH ASSOCIATE PROFESSIONALS'
 'SCIENCE AND ENGINEERING ASSOCIATE PROFESSIONALS' 'HEALTH PROFESSIONALS'
 'MARKET-ORIENTED SKILLED AGRICULTURAL WORKERS'
 'DRIVERS AND MOBILE PLANT OPERATORS'
 'BUSINESS AND ADMINISTRATION PROFESSIONALS' 'SALES WORKERS'
 'INFORMATION AND COMMUNICATIONS TECHNICIANS' 'CUSTOMER SERVICES CLERKS'
 'INFORMATION AND COMMUNICATIONS TECHNOLOGY PROFESSIONALS'
 'PROTECTIVE SERVICES WORKERS' 'ADMINISTRATIVE AND COMMERCIAL MANAGERS'
 'CLEANERS AND HELPERS'] 16
[21 33 23 32 31 22 61 83 24 52 35 42 25 54 12 91] 16
job
supervisor             84
inspection engineer    72
veterinary surgeon     57
bus driver             55
pharmacist             54
                       ..
immunologist           10
lender                 10
speech therapist       10
system programmer      10
nutritionist           10
Name: count, Length: 159, dtype: int64


In [5]:
for i in range(9):
    print(" ".join(df.iloc[i].position.split(" ")[:-1]),'\t', df.iloc[i].job)
    # print(df.iloc[i].description[3:-3].replace(". ", ".\n"))
    # print()

Accelerator Physicist 	 physicist
Applied Physicist (Computing) (EP-LBC-2021-125-LD) 	 physicist
Accelerator Physicist (BE-ABP-LNO-2021-122-LD) 	 physicist
Medical Devices 	 physicist
Fluidics 	 physicist
Graduate Experimental Engineering 	 physicist
Magnet Engineer / Senior Magnet Engineer / 	 physicist
Magnet Engineer / Senior Magnet Engineer / 	 physicist
Magnet Engineer / Senior Magnet Engineer / 	 physicist


## Resume

In [6]:
path = kagglehub.dataset_download("snehaanbhawal/resume-dataset")

print("Path to dataset files:", path)

Path to dataset files: /Users/socialaistudio/Code/hiring_agent/data/kagglehub/datasets/snehaanbhawal/resume-dataset/versions/1


In [7]:
resume_df = pd.read_csv(path + "/Resume/resume.csv")
resume_df.drop(columns=['ID', 'Resume_html'], inplace=True)
resume_df.rename(columns={'Resume_str': 'resume'}, inplace=True)
resume_df = resume_df[['Category', 'resume']]
resume_df.head()

,Category,resume
0,HR,HR ADMINISTRATOR/MARKETING ASSOCIATE\...
1,HR,"HR SPECIALIST, US HR OPERATIONS ..."
2,HR,HR DIRECTOR Summary Over 2...
3,HR,HR SPECIALIST Summary Dedica...
4,HR,HR MANAGER Skill Highlights ...


In [8]:
# resume_df.iloc[0].resume
resume_df.Category.unique(), len(resume_df.Category.unique())

(array(['HR', 'DESIGNER', 'INFORMATION-TECHNOLOGY', 'TEACHER', 'ADVOCATE',
        'BUSINESS-DEVELOPMENT', 'HEALTHCARE', 'FITNESS', 'AGRICULTURE',
        'BPO', 'SALES', 'CONSULTANT', 'DIGITAL-MEDIA', 'AUTOMOBILE',
        'CHEF', 'FINANCE', 'APPAREL', 'ENGINEERING', 'ACCOUNTANT',
        'CONSTRUCTION', 'PUBLIC-RELATIONS', 'BANKING', 'ARTS', 'AVIATION'],
       dtype=object),
 24)

In [12]:
from src.utils.pipeline_utils import process_resume_pipeline
import time

curr_time = time.strftime("%Y%m%d-%H%M%S")

resume_dct = process_resume_pipeline(resume_df.iloc[0]['resume'], country="Singapore")
# write the results to markdown files
for k, v in resume_dct.items():
    os.makedirs(f"results/{curr_time}", exist_ok=True)
    with open(f"results/{curr_time}/{k}_resume.md", "w") as f:
        f.write(v)

2025-09-12 18:02:49,990 - AnonymizationAgent - INFO - Starting resume anonymization...
2025-09-12 18:03:28,060 - AnonymizationAgent - INFO - Resume anonymization completed successfully.
2025-09-12 18:03:28,061 - LocalizationPipeline - INFO - Anonymized resume:
 ## Anonymized Resume

HR ADMINISTRATOR/MARKETING ASSOCIATE

**Summary**  
Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management. Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.

**Highlights**  
- Focused on customer satisfaction  
- Team management  
- Marketing savvy  
- Conflict resolution techniques  
- Training and development  
- Skilled multi-tasker  
- Client relations specialist  

**Accomplishments**  
- Missouri DOT Supervisor Training Certification  
- Certified by [COMPANY X] in Customer Loyalty and Marketing by Segment  
- [COMPANY X] General Manager Training Certification  
- A

In [108]:

# create an LLM agent
from langchain.agents import LLMSingleActionAgent
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

# initialize the agent
llm = OpenAI(temperature=0.0,
            model="gpt-4o-mini",
            base_url="https://openrouter.ai/api/v1",
            api_key=os.environ.get("OPENROUTER_API_KEY"))

# define the agent's instruction
prompt = """Given a list of resume and job categories, assign which resume category aligns with the job category.

Resume Categories:
{resume_categories}

Job Categories:
{job_categories}

Provide your answer in the format:
resume_category: job_category

Do not generate additional text
"""

# add the variables to the agent
prompt = PromptTemplate.from_template(prompt)

prompt.invoke({
    "resume_categories": "Software Engineer, Data Scientist, Marketing Manager",
    "job_categories": "Software Engineer, Data Scientist, Marketing Manager"
})
    

StringPromptValue(text='Given a list of resume and job categories, assign which resume category aligns with the job category.\n\nResume Categories:\nSoftware Engineer, Data Scientist, Marketing Manager\n\nJob Categories:\nSoftware Engineer, Data Scientist, Marketing Manager\n\nProvide your answer in the format:\nresume_category: job_category\n\nDo not generate additional text\n')

In [109]:
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser
from langchain.schema.runnable import Runnable

llm = ChatOpenAI(
    model="gpt-4o-mini",
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ.get("OPENROUTER_API_KEY"),
    temperature=0.0,
)

chain: Runnable = prompt | llm | StrOutputParser()

output = chain.invoke({"resume_categories": list(resume_df.Category.unique()), "job_categories": list(df['major_job'].unique())})

In [110]:
print(output)

HR: BUSINESS AND ADMINISTRATION PROFESSIONALS  
DESIGNER: SCIENCE AND ENGINEERING PROFESSIONALS  
INFORMATION-TECHNOLOGY: INFORMATION AND COMMUNICATIONS TECHNOLOGY PROFESSIONALS  
TEACHER: TEACHING PROFESSIONALS  
ADVOCATE: PROTECTIVE SERVICES WORKERS  
BUSINESS-DEVELOPMENT: BUSINESS AND ADMINISTRATION ASSOCIATE PROFESSIONALS  
HEALTHCARE: HEALTH PROFESSIONALS  
FITNESS: HEALTH ASSOCIATE PROFESSIONALS  
AGRICULTURE: MARKET-ORIENTED SKILLED AGRICULTURAL WORKERS  
BPO: CUSTOMER SERVICES CLERKS  
SALES: SALES WORKERS  
CONSULTANT: ADMINISTRATIVE AND COMMERCIAL MANAGERS  
DIGITAL-MEDIA: INFORMATION AND COMMUNICATIONS TECHNICIANS  
AUTOMOBILE: DRIVERS AND MOBILE PLANT OPERATORS  
CHEF: CLEANERS AND HELPERS  
FINANCE: BUSINESS AND ADMINISTRATION PROFESSIONALS  
APPAREL: SCIENCE AND ENGINEERING ASSOCIATE PROFESSIONALS  
ENGINEERING: SCIENCE AND ENGINEERING PROFESSIONALS  
ACCOUNTANT: BUSINESS AND ADMINISTRATION PROFESSIONALS  
CONSTRUCTION: SCIENCE AND ENGINEERING PROFESSIONALS  
PUBLIC-RELAT

In [111]:
mapping_dct = {x:y for x, y in map(lambda v: v.strip().split(": "), output.split("\n"))}
mapping_dct

{'HR': 'BUSINESS AND ADMINISTRATION PROFESSIONALS',
 'DESIGNER': 'SCIENCE AND ENGINEERING PROFESSIONALS',
 'INFORMATION-TECHNOLOGY': 'INFORMATION AND COMMUNICATIONS TECHNOLOGY PROFESSIONALS',
 'TEACHER': 'TEACHING PROFESSIONALS',
 'ADVOCATE': 'PROTECTIVE SERVICES WORKERS',
 'BUSINESS-DEVELOPMENT': 'BUSINESS AND ADMINISTRATION ASSOCIATE PROFESSIONALS',
 'HEALTHCARE': 'HEALTH PROFESSIONALS',
 'FITNESS': 'HEALTH ASSOCIATE PROFESSIONALS',
 'AGRICULTURE': 'MARKET-ORIENTED SKILLED AGRICULTURAL WORKERS',
 'BPO': 'CUSTOMER SERVICES CLERKS',
 'SALES': 'SALES WORKERS',
 'CONSULTANT': 'ADMINISTRATIVE AND COMMERCIAL MANAGERS',
 'DIGITAL-MEDIA': 'INFORMATION AND COMMUNICATIONS TECHNICIANS',
 'AUTOMOBILE': 'DRIVERS AND MOBILE PLANT OPERATORS',
 'CHEF': 'CLEANERS AND HELPERS',
 'FINANCE': 'BUSINESS AND ADMINISTRATION PROFESSIONALS',
 'APPAREL': 'SCIENCE AND ENGINEERING ASSOCIATE PROFESSIONALS',
 'ENGINEERING': 'SCIENCE AND ENGINEERING PROFESSIONALS',
 'ACCOUNTANT': 'BUSINESS AND ADMINISTRATION PROFES

## Run

In [131]:
import random
random.seed(42)

# Choose a random job description and resume for each resume_category

test_set = []

for resume_cat, job in mapping_dct.items():
    random_resume = resume_df[resume_df['Category'] == resume_cat].sample(n = 1, random_state=42).iloc[0]['resume'].strip()
    random_job = df[df['major_job'] == job].sample(n = 1, random_state=42).iloc[0]['description'][3:-3]
    test_set.append((random_job, random_resume))

test_set[-1]
    

("Refuel the vehicles on their return to the depot Sweep the vehicle to a high standard Wash the vehicle – using the mechanical wash Park the vehicle safely in the depot ', ' Hold a full car licence Ideally hold a PCV or LGV licence",
 "AIRCRAFT SALES       Professional Summary    I expect to associate myself with a professionally driven, well-respected organization by pursuing a challenging, responsible, dynamic and rewarding career and applying my academic knowledge, expertise, skills and enthusiasm; which will assist my career development and contribute effectively to the progress of the organization and our society as well. Areas of expertise include investigative policy research and Statistical Analysis, proven ability to interact effectively with employee and the public in a professional manner. Great team leader with strong organizational skills and formidable work ethic. Seventeen years construction management experience, with industrial and commercial construction jobs valued 